In [ ]:
! pip install openai
! pip install chromadb
! pip install langchain-google-genai
! pip install tiktoken
! pip install langchain
! pip install pinecone-client
! pip install pypdf

In [ ]:
import langchain
from langchain.embeddings import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.conversation.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryBufferMemory
)
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
import os
import getpass


In [ ]:
PINECONE_API_KEY=getpass.getpass('Put Pinecone API Here')

Put Pinecone API Here··········


In [ ]:
OPENAI_API_KEY=getpass.getpass('Openai Key Here')

Openai Key Here··········


In [ ]:
!mkdir pdf

mkdir: cannot create directory ‘pdf’: File exists


In [ ]:
loader=PyPDFDirectoryLoader('/content/pdf',glob='./*.pdf')

In [ ]:
documents=loader.load()

In [ ]:
def get_chunks(documents,chunk_size:int=2000,chunk_overlap:int=500):
  d_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
  return d_splitter.split_documents(documents)

In [ ]:
chunks=get_chunks(documents)

In [ ]:
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

In [ ]:
embedding=OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
PINECONE_API_ENV=os.environ.get('PINECONE_API_ENV','gcp-starter')

In [ ]:
embed=embedding.embed_query('Hello World')

In [ ]:
len(embed)

1536

In [ ]:
from pinecone import Pinecone
pc=Pinecone(
    api_key=PINECONE_API_KEY
)


In [ ]:
index_name='pinecone'

In [ ]:
index=pc.Index(index_name)

In [ ]:
chunks

In [ ]:
import uuid

In [ ]:
for i,chunk in enumerate(chunks):
  metadatas=[{
      'title':chunk.metadata['source'],
      'text':chunk.page_content
  }]
  unique_id = str(uuid.uuid4())
  embed=embedding.embed_documents(chunk.page_content)
  index.upsert(vectors=zip(unique_id,embed,metadatas))


In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00013,
 'namespaces': {'': {'vector_count': 13}},
 'total_vector_count': 13}

In [ ]:
text_field='text'

In [ ]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embedding.embed_query, text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [ ]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-west-2"
)

In [ ]:
import time

index_name = "langchain-retrieval-agent"
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of ada 002
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 9}},
 'total_vector_count': 9}

In [ ]:
for i,chunk in enumerate(chunks):
  metadatas=[{
      'title':chunk.metadata['source'],
      'text':chunk.page_content
  }]
  unique_id = str(uuid.uuid4())
  embed=embedding.embed_documents(chunk.page_content)
  index.upsert(vectors=zip(unique_id,embed,metadatas))

In [ ]:
from langchain.vectorstores import Pinecone
text_field = "text"
vectordb=Pinecone(
    index,
    embedding.embed_query,
    text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [ ]:
# initialize the vector store object
query = "In Which University did Fayaz Ali apply?"

vectordb.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

In [ ]:
retriever=vectordb.as_retriever()

In [ ]:
qa_bot=RetrievalQA.from_chain_type(llm=OpenAI(),chain_type='stuff',retriever=retriever,return_source_documents=True)

In [ ]:
res=qa_bot('In project Fine-tune LLMs and build your application. What is required by me? Give me all the details')

In [ ]:
res

{'query': 'In project Fine-tune LLMs and build your application. What is required by me? Give me all the details',
 'result': ' In the project Fine-tune LLMs and build your application, you are required to establish a comprehensive framework that streamlines the fine-tuning process of Large Language Models (LLMs) and provides a user-friendly interface for building applications on top of these tailored language models. This includes addressing key issues such as integration complexity, fine-tuning accessibility, and application development barriers. You will also need to curate a diverse dataset specific to the Indian continent and maintain your code on GitHub. Additionally, you must use a given dataset (MongoDB and MySQL) and can choose to host your solution on a cloud platform such as AWS, Azure, or GCP. Finally, you must expose your solution as an API or create a user interface for model testing. ',
 'source_documents': [Document(page_content='1   \n \n  \n \n \n \nProject  Title Fin